# Talking to other processors over USB-C

*Connecting a microprocessor to the Create*

## A first test

To succeed here you need to know your Create's IP address ([here](https://docs.google.com/spreadsheets/d/1xecZjXfFmFzZ6gqWdvJEo7XQHoThx7uRky23N8nAhvs/edit?usp=sharing)) and set up the serial and port.
1. type the IP address into a web browser to get to the Create's webpage
2. go to IPaddress/serial-config
3. TTY device to forward: ttyACM0
4. Baud rate: 115200
5. External port number: 8883 (that is teh only one that works)
6. hit save
7. restart your robot
8. connect up your MakerPi to the Create
9. try running the code below - do not forget to change the IP address and the name of your create - can you figure out what it does?


In [ ]:
from Subs.CreateLib import Create
import time

CREATE_IP = "192.168.86.31"
CREATE_PORT = 8883

code = '''
import time

def MakerPi():
    for i in range(10):
        print(i)
        time.sleep(1)
        
MakerPi()
'''

def main():
    MyCreate = Create('/Buzz')
    MyCreate.serial_init(CREATE_IP, CREATE_PORT, 1)
    MyCreate.serial_abort() # stop any program that is running
    MyCreate.serial_write('2 + 2 \r\n') # direct command
    print(MyCreate.serial_read())
    MyCreate.serial_run(code)    # download and run a code
    print(MyCreate.serial_read())
    try:
        for i in range(10):
            time.sleep(1)
            print('iteration %d, read %s' % (i,MyCreate.serial_read()))
    except Exception as e:
        print(e)
    MyCreate.serial_abort()
    MyCreate.serial_close()
    MyCreate.close()
    print('done')
        
main()


If you get a *Connection Refused* error, that probably means you are on BLE mode rather than USB mode - so flip the switch.  

## Understanding the code

The first thing is define your IP addresses and to figure out what code you want running on the MakerPi - that goes in the variable code:

In [ ]:
from Subs.CreateLib import Create
import time

CREATE_IP = "192.168.86.225"
CREATE_PORT = 8883

code = '''
import time

def MakerPi():
    for i in range(10):
        print(i)
        time.sleep(1)
        
MakerPi()
'''

Then you connect up to the serial port (using TCP/IP) and that gets you to the REPL on the MakerPi.  First send a CtrlC (abort) to stop any code that is currently running.  Then ask it what *2+2* is - the *\r\n* are carriage return and line feed - needed on every line for micropython.  And then you read back the answer and print it out.

In [ ]:
MyCreate = Create('/rogers')
MyCreate.serial_init(CREATE_IP, CREATE_PORT, 1)
MyCreate.serial_abort() # stop any program that is running
MyCreate.serial_write('2 + 2 \r\n') # direct command
time.sleep(1)
print(MyCreate.serial_read())

Then you load up the code - and it runs because you call *MakerPi()* right away - and you read back what it prints out and show it here.

In [ ]:
MyCreate.serial_run(code)
print(MyCreate.serial_read())
try:
    for i in range(10):
        time.sleep(1)
        print('iteration %d, read %s' % (i,MyCreate.serial_read()))
except Exception as e:
    print(e)

Then you close all open connections and say you are done

In [ ]:
MyCreate.serial_abort()
MyCreate.serial_close()
MyCreate.close()
print('done')

and you are done - now you can put in whatever code you want and control the MakerPi from your code.  Note if you stop code from running, you will have to reset the kernel to run it again (since the MyCreate.close() never was run).  Note that the code upload has not been extensively tested - so I recommend using libraries when possible to make your code short.